In [7]:
import pandas as pd
import numpy as np

# Assuming we start with your VAM solution
# We'll use the allocation matrix from VAM as our initial solution

import pandas as pd
import numpy as np

# Load Distance matrix
distance_df = pd.read_csv("Data.csv", index_col=0)

# Define demand
demand = {
    'Guwhathi': 26, 'Srinagar': 32.1, 'Chandigarh': 28.6, 'Delhi': 244.3,
    'Rudrapur': 38, 'NaviMumbai': 287.7, 'Vijaywada': 27.7, 'Varanasi': 32.6,
    'Kharagpur': 21.7, 'Indore': 54.3, 'Belgaum': 43.4, 'Bangalore': 211.7,
    'Rajkot': 38, 'Ahmedabad': 149.3, 'Ludhiana': 43.4, 'Rourkela': 40.2,
    'Hyderabad': 184.8, 'Nagpur': 65.1, 'Coimbatore': 43.4, 'Kolkata': 121.9
}

# Define supply
supply = {
    'Kolkata': 31.6, 'Haldia': 51, 'Paradeep': 239, 'Vizag': 131.1, 'Chennai': 225,
    'Chidambaram': 331.5, 'Kochi': 78.9, 'Mangalore': 98, 'Mormugaon': 63.4,
    'JNPT': 217.6, 'DeenDayal': 267.1, 'Kharagpur': 0, 'Chandigarh': 0, 'Delhi': 0, 'Bangalore': 0
}

# Filter out sources and destinations with zero supply/demand
supply = {k: v for k, v in supply.items() if v > 0}
demand = {k: v for k, v in demand.items() if v > 0}

# Ensure distance_df only includes relevant sources and destinations
cost_matrix = distance_df.loc[supply.keys(), demand.keys()].fillna(2000)

# Initialize variables for VAM
supply_vals = supply.copy()
demand_vals = demand.copy()
allocation = pd.DataFrame(0, index=supply.keys(), columns=demand.keys())

# Function to calculate penalties
def calculate_penalties(matrix, axis=0):
    """
    Calculate penalties for rows (axis=0) or columns (axis=1).
    """
    if axis == 0:  # Row-wise
        penalties = []
        for row in matrix.iterrows():
            row_data = row[1].sort_values()
            penalties.append(row_data.iloc[1] - row_data.iloc[0] if len(row_data) > 1 else 0)
        return penalties
    else:  # Column-wise
        penalties = []
        for col in matrix.columns:
            col_data = matrix[col].sort_values()
            penalties.append(col_data.iloc[1] - col_data.iloc[0] if len(col_data) > 1 else 0)
        return penalties

# Vogel's Approximation Method
while supply_vals and demand_vals:
    # Calculate penalties for both rows and columns
    row_penalties = calculate_penalties(cost_matrix, axis=0)
    col_penalties = calculate_penalties(cost_matrix, axis=1)

    # Find the highest penalty
    max_penalty_row = np.argmax(row_penalties)
    max_penalty_col = np.argmax(col_penalties)

    # If the row penalty is higher, we select the row, else select the column
    if row_penalties[max_penalty_row] > col_penalties[max_penalty_col]:
        i = cost_matrix.index[max_penalty_row]
        j = cost_matrix.loc[i].idxmin()  # Select the minimum cost in the row
    else:
        j = cost_matrix.columns[max_penalty_col]
        i = cost_matrix[j].idxmin()  # Select the minimum cost in the column

    # Allocate as much as possible to the selected cell
    alloc = min(supply_vals[i], demand_vals[j])
    allocation.loc[i, j] = alloc

    # Update supply and demand
    supply_vals[i] -= alloc
    demand_vals[j] -= alloc

    # If supply at i is exhausted, remove it from the supply dictionary and cost matrix
    if supply_vals[i] == 0:
        del supply_vals[i]
        cost_matrix = cost_matrix.drop(i)

    # If demand at j is exhausted, remove it from the demand dictionary and cost matrix
    if demand_vals[j] == 0:
        del demand_vals[j]
        cost_matrix = cost_matrix.drop(columns=[j])

    # Recalculate the cost matrix after the update
    cost_matrix = cost_matrix.loc[supply_vals.keys(), demand_vals.keys()]

# Calculate total cost
total_cost = (allocation * distance_df.loc[allocation.index, allocation.columns]).sum().sum()

# Print output
print("Initial Basic Feasible Solution (Allocation Matrix):")
print(allocation)
print(f"\nTotal Transportation Cost: {35 * total_cost}")

# Round to integer and save
allocation_rounded = allocation.round(0).astype(int)

# Save the rounded allocation matrix to CSV
allocation_rounded.to_csv("VAM_Allocation.csv")


def find_closed_loop(allocation, start_i, start_j, cost_matrix):
    """
    Find a closed loop starting from position (start_i, start_j)
    Returns list of coordinates forming the loop
    """
    rows, cols = allocation.shape
    used_cells = [(i, j) for i in range(rows) for j in range(cols) if allocation.iloc[i, j] > 0]
    loop = [(start_i, start_j)]
    direction = 'horizontal'  # Start by looking horizontally

    while True:
        curr_i, curr_j = loop[-1]

        if direction == 'horizontal':
            # Look for next vertical step
            next_cells = [(i, curr_j) for i, j in used_cells if j == curr_j and i != curr_i]
            if not next_cells:
                return None  # No valid loop found
            next_i, next_j = next_cells[0]
            direction = 'vertical'

        else:  # vertical
            # Look for next horizontal step
            next_cells = [(i, j) for i, j in used_cells if i == curr_i and j != curr_j]
            if not next_cells:
                return None  # No valid loop found
            next_i, next_j = next_cells[0]
            direction = 'horizontal'

        if (next_i, next_j) in loop:
            if (next_i, next_j) == (start_i, start_j) and len(loop) >= 4 and len(loop) % 2 == 0:
                return loop
            return None

        loop.append((next_i, next_j))

def stepping_stone_method(cost_matrix, initial_allocation):
    """
    Implement Stepping Stone Method to optimize the initial solution
    """
    allocation = initial_allocation.copy()
    iteration = 0

    while True:
        iteration += 1
        print(f"\nIteration {iteration}:")

        # Find all unoccupied cells
        rows, cols = allocation.shape
        unoccupied = [(i, j) for i in range(rows) for j in range(cols) if allocation.iloc[i, j] == 0]

        # Calculate opportunity cost for each unoccupied cell
        min_opportunity_cost = float('inf')
        best_loop = None
        best_cell = None

        for i, j in unoccupied:
            loop = find_closed_loop(allocation, i, j, cost_matrix)
            if loop is None:
                continue

            # Calculate opportunity cost
            opportunity_cost = 0
            for k, (loop_i, loop_j) in enumerate(loop):
                sign = -1 if k % 2 == 0 else 1
                opportunity_cost += sign * cost_matrix.iloc[loop_i, loop_j]

            if opportunity_cost < min_opportunity_cost:
                min_opportunity_cost = opportunity_cost
                best_loop = loop
                best_cell = (i, j)

        # If no negative opportunity cost found, solution is optimal
        if min_opportunity_cost >= 0:
            print("Optimal solution found!")
            break

        # Adjust allocation along the loop
        min_adjust = float('inf')
        for k, (i, j) in enumerate(best_loop):
            if k % 2 == 1:  # Negative positions
                min_adjust = min(min_adjust, allocation.iloc[i, j])

        # Update allocation
        for k, (i, j) in enumerate(best_loop):
            sign = -1 if k % 2 == 0 else 1
            allocation.iloc[i, j] += sign * min_adjust

        print(f"Adjusted allocation at cell {best_cell} with opportunity cost {min_opportunity_cost}")
        print("Current Allocation:")
        print(allocation)

    # Calculate total cost
    total_cost = (allocation * cost_matrix.loc[allocation.index, allocation.columns]).sum().sum()
    return allocation, total_cost

# Using your existing data
# Execute VAM first to get initial_allocation (your allocation_rounded)
initial_allocation = allocation_rounded  # From your VAM solution
cost_matrix = distance_df.loc[supply.keys(), demand.keys()].fillna(2000)

# Run Stepping Stone Method
optimal_allocation, optimal_cost = stepping_stone_method(cost_matrix, initial_allocation)

# Print results
print("\nFinal Optimal Allocation:")
print(optimal_allocation)
print(f"Final Total Transportation Cost: {35 * optimal_cost}")

# Save the optimal solution
optimal_allocation.to_csv("Stepping_Stone_Allocation.csv")

<ipython-input-7-1455b3f57091>:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  allocation.loc[i, j] = alloc
<ipython-input-7-1455b3f57091>:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '217.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  allocation.loc[i, j] = alloc
<ipython-input-7-1455b3f57091>:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '149.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  allocation.loc[i, j] = alloc
<ipython-input-7-1455b3f57091>:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a fut

Initial Basic Feasible Solution (Allocation Matrix):
             Guwhathi  Srinagar  Chandigarh  Delhi  Rudrapur  NaviMumbai  \
Kolkata             0       0.0         0.0    0.0         0         0.0   
Haldia              0       0.0         0.0    0.0         0         0.0   
Paradeep           26      32.1         0.0    0.9        38         0.0   
Vizag               0       0.0         0.0    0.0         0         0.0   
Chennai             0       0.0         0.0    0.0         0         0.0   
Chidambaram         0       0.0         0.0  243.4         0        50.1   
Kochi               0       0.0         0.0    0.0         0         0.0   
Mangalore           0       0.0         0.0    0.0         0         0.0   
Mormugaon           0       0.0         0.0    0.0         0        20.0   
JNPT                0       0.0         0.0    0.0         0       217.6   
DeenDayal           0       0.0        28.6    0.0         0         0.0   

             Vijaywada  Varanasi  

In [9]:
import pandas as pd
import numpy as np

# Load Distance matrix
distance_df = pd.read_csv("Data.csv", index_col=0)

# Define demand
demand = {
    'Guwhathi': 26, 'Srinagar': 32.1, 'Chandigarh': 28.6, 'Delhi': 244.3,
    'Rudrapur': 38, 'NaviMumbai': 287.7, 'Vijaywada': 27.7, 'Varanasi': 32.6,
    'Kharagpur': 21.7, 'Indore': 54.3, 'Belgaum': 43.4, 'Bangalore': 211.7,
    'Rajkot': 38, 'Ahmedabad': 149.3, 'Ludhiana': 43.4, 'Rourkela': 40.2,
    'Hyderabad': 184.8, 'Nagpur': 65.1, 'Coimbatore': 43.4, 'Kolkata': 121.9
}

# Define supply
supply = {
    'Kolkata': 31.6, 'Haldia': 51, 'Paradeep': 239, 'Vizag': 131.1, 'Chennai': 225,
    'Chidambaram': 331.5, 'Kochi': 78.9, 'Mangalore': 98, 'Mormugaon': 63.4,
    'JNPT': 217.6, 'DeenDayal': 267.1, 'Kharagpur': 0, 'Chandigarh': 0, 'Delhi': 0, 'Bangalore': 0
}

# Filter out sources and destinations with zero supply/demand
supply = {k: v for k, v in supply.items() if v > 0}
demand = {k: v for k, v in demand.items() if v > 0}

# Ensure distance_df only includes relevant sources and destinations
original_cost_matrix = distance_df.loc[supply.keys(), demand.keys()].fillna(2000)

# Initialize variables for VAM
supply_vals = supply.copy()
demand_vals = demand.copy()
allocation = pd.DataFrame(0, index=supply.keys(), columns=demand.keys())
cost_matrix = original_cost_matrix.copy()  # Keep a working copy for VAM

# Function to calculate penalties
def calculate_penalties(matrix, axis=0):
    if axis == 0:  # Row-wise
        penalties = []
        for row in matrix.iterrows():
            row_data = row[1].sort_values()
            penalties.append(row_data.iloc[1] - row_data.iloc[0] if len(row_data) > 1 else 0)
        return penalties
    else:  # Column-wise
        penalties = []
        for col in matrix.columns:
            col_data = matrix[col].sort_values()
            penalties.append(col_data.iloc[1] - col_data.iloc[0] if len(col_data) > 1 else 0)
        return penalties

# Vogel's Approximation Method
while supply_vals and demand_vals:
    row_penalties = calculate_penalties(cost_matrix, axis=0)
    col_penalties = calculate_penalties(cost_matrix, axis=1)

    max_penalty_row = np.argmax(row_penalties)
    max_penalty_col = np.argmax(col_penalties)

    if row_penalties[max_penalty_row] > col_penalties[max_penalty_col]:
        i = cost_matrix.index[max_penalty_row]
        j = cost_matrix.loc[i].idxmin()
    else:
        j = cost_matrix.columns[max_penalty_col]
        i = cost_matrix[j].idxmin()

    alloc = min(supply_vals[i], demand_vals[j])
    allocation.loc[i, j] = alloc

    supply_vals[i] -= alloc
    demand_vals[j] -= alloc

    if supply_vals[i] == 0:
        del supply_vals[i]
        cost_matrix = cost_matrix.drop(i)

    if demand_vals[j] == 0:
        del demand_vals[j]
        cost_matrix = cost_matrix.drop(columns=[j])

    cost_matrix = cost_matrix.loc[supply_vals.keys(), demand_vals.keys()]

# Calculate total cost
total_cost = (allocation * original_cost_matrix).sum().sum()

# Print output
print("Initial Basic Feasible Solution (Allocation Matrix):")
print(allocation)
print(f"\nTotal Transportation Cost: {35 * total_cost}")

# Round to integer and save
allocation_rounded = allocation.round(0).astype(int)
allocation_rounded.to_csv("VAM_Allocation.csv")

def modi_method(cost_matrix, initial_allocation):
    allocation = initial_allocation.copy()
    cost_matrix = cost_matrix.loc[allocation.index, allocation.columns].astype(float)

    while True:
        # Step 1: Calculate u and v values
        rows, cols = allocation.shape
        u = [None] * rows
        v = [None] * cols

        u[0] = 0
        basic_cells = [(i, j) for i in range(rows) for j in range(cols) if allocation.iloc[i, j] > 0]

        while None in u or None in v:
            for i, j in basic_cells:
                if u[i] is not None and v[j] is None:
                    v[j] = cost_matrix.iloc[i, j] - u[i]
                elif v[j] is not None and u[i] is None:
                    u[i] = cost_matrix.iloc[i, j] - v[j]

        # Step 2: Calculate opportunity costs for non-basic cells
        opportunity_costs = pd.DataFrame(index=allocation.index, columns=allocation.columns)
        for i in range(rows):
            for j in range(cols):
                if allocation.iloc[i, j] == 0:
                    opportunity_costs.iloc[i, j] = cost_matrix.iloc[i, j] - (u[i] + v[j])
                else:
                    opportunity_costs.iloc[i, j] = 0

        # Step 3: Check optimality
        min_opportunity_cost = opportunity_costs.min().min()
        if min_opportunity_cost >= 0:
            print("Optimal solution found!")
            break

        # Step 4: Find entering cell
        entering_i, entering_j = np.unravel_index(opportunity_costs.values.argmin(), opportunity_costs.shape)
        entering_i = opportunity_costs.index[entering_i]
        entering_j = opportunity_costs.columns[entering_j]

        # Step 5: Find closed loop
        def find_loop(allocation, start_i, start_j):
            used_cells = [(i, j) for i in allocation.index for j in allocation.columns
                         if allocation.loc[i, j] > 0]
            loop = [(start_i, start_j)]
            direction = 'horizontal'

            while True:
                curr_i, curr_j = loop[-1]
                if direction == 'horizontal':
                    next_cells = [(i, curr_j) for i, j in used_cells if j == curr_j and i != curr_i]
                    if not next_cells:
                        return None
                    next_i, next_j = next_cells[0]
                    direction = 'vertical'
                else:
                    next_cells = [(i, j) for i, j in used_cells if i == curr_i and j != curr_j]
                    if not next_cells:
                        return None
                    next_i, next_j = next_cells[0]
                    direction = 'horizontal'

                if (next_i, next_j) in loop:
                    if (next_i, next_j) == (start_i, start_j) and len(loop) >= 4 and len(loop) % 2 == 0:
                        return loop
                    return None
                loop.append((next_i, next_j))

        loop = find_loop(allocation, entering_i, entering_j)
        if loop is None:
            print("No valid loop found. Solution may be degenerate.")
            break

        # Step 6: Adjust allocation
        min_adjust = float('inf')
        for k, (i, j) in enumerate(loop):
            if k % 2 == 1:
                min_adjust = min(min_adjust, allocation.loc[i, j])

        for k, (i, j) in enumerate(loop):
            sign = -1 if k % 2 == 0 else 1
            allocation.loc[i, j] += sign * min_adjust

        print(f"Adjusted allocation at {entering_i}-{entering_j} with opportunity cost {min_opportunity_cost}")
        print("Current Allocation:")
        print(allocation)

    total_cost = (allocation * cost_matrix).sum().sum()
    return allocation, total_cost

# Run MODI
print("\nStarting MODI Method Optimization...")
initial_allocation = allocation_rounded
optimal_allocation, optimal_cost = modi_method(original_cost_matrix, initial_allocation)

# Print final results
print("\nFinal Optimal Allocation:")
print(optimal_allocation)
print(f"Final Total Transportation Cost: {35 * optimal_cost}")

# Save the optimal solution
optimal_allocation.to_csv("MODI_Allocation.csv")

Initial Basic Feasible Solution (Allocation Matrix):
             Guwhathi  Srinagar  Chandigarh  Delhi  Rudrapur  NaviMumbai  \
Kolkata             0       0.0         0.0    0.0         0         0.0   
Haldia              0       0.0         0.0    0.0         0         0.0   
Paradeep           26      32.1         0.0    0.9        38         0.0   
Vizag               0       0.0         0.0    0.0         0         0.0   
Chennai             0       0.0         0.0    0.0         0         0.0   
Chidambaram         0       0.0         0.0  243.4         0        50.1   
Kochi               0       0.0         0.0    0.0         0         0.0   
Mangalore           0       0.0         0.0    0.0         0         0.0   
Mormugaon           0       0.0         0.0    0.0         0        20.0   
JNPT                0       0.0         0.0    0.0         0       217.6   
DeenDayal           0       0.0        28.6    0.0         0         0.0   

             Vijaywada  Varanasi  

<ipython-input-9-caaebb405537>:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  allocation.loc[i, j] = alloc
<ipython-input-9-caaebb405537>:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '217.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  allocation.loc[i, j] = alloc
<ipython-input-9-caaebb405537>:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '149.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  allocation.loc[i, j] = alloc
<ipython-input-9-caaebb405537>:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a fut